In [ ]:
%matplotlib inline
import ast
import json
import urllib
import urllib2
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cStringIO import StringIO

We define a helper function <b>queryCartoDB</b> to wrap around the SQL API of CartoDB. We can simply pass in a public data source (in the form of 'https://{ACCOUNT}.cartodb.com/api/v2/sql', where ACCOUNT is the owner of the data) and a SQL statement for us to query the data in various formats including CSV, JSON, and GeoJSON. The result returned from this function is a file-like object, where we can pass to file readers such as Pandas' <b>read_csv()</b> or the JSON parser.

In our example, <b>source</b> is assigned a citibike data set available on the htv210 account.

We also assign <b>query1</b> and <b>query2</b> as the sample solution of homework 7.

In [ ]:
def queryCartoDB(source, query, format='CSV'):
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

source = 'https://htv210.cartodb.com/api/v2/sql'
query1 = open('hw7_1.sql', 'r').read()
query2 = open('hw7_2.sql', 'r').read()

We are just playing around with SQL statements in the next step. Note that, data can be retrieved in any format that we prefer.

In [ ]:
query = '''
SELECT *
FROM citibike
LIMIT 10
'''

result = queryCartoDB(source, query, 'CSV')
df     = pd.read_csv(result)
df.head()

For the second query, where we would like visualize the output geometries, we use the Python's JSON module to parse the data and feed it to a GeoDataFrame. We can see the result in a plot

In [ ]:
result = queryCartoDB(source, query2, 'GeoJSON')
data   = json.loads(result.read())
gdata  = gpd.GeoDataFrame.from_features(data['features'])

In [ ]:
f, ax = plt.subplots(1, figsize=(5,10))
gdata.plot(column='trip_count', colormap='OrRd', alpha=1, axes=ax, scheme='QUANTILES', k=9)

To make the dots larger, we could create a buffer at each point (200 ft radius) and reassign the GeoDataFrame geometries.

In [ ]:
dots   = gdata.set_geometry(gdata.geometry.buffer(100))

In [ ]:
f, ax = plt.subplots(1, figsize=(5,10))
dots.plot(column='trip_count', colormap='OrRd', alpha=1, axes=ax, scheme='QUANTILES', k=9)